In [13]:
import json
import requests
import csv
import jsonlines
from flask import Flask, redirect, render_template, request, url_for
import sys
import numpy as np
from collections import defaultdict
import random
import math
import openai
import re
import pandas as pd
#csv.field_size_limit(sys.maxsize)

In [14]:
def save_json(data, filepath=r'new_data.json'):
    with open(filepath, 'w') as fp:
        json.dump(data, fp, indent=4)

In [15]:
openai.api_key = ''

In [16]:
data = csv.DictReader(open("D:\Projects\Test\IEEE_papers\Raw_data\IEEE VIS papers 1990-2022 - Main dataset.csv"))

In [17]:
dataset = [datum for datum in data]
dataset = dataset[0:20]

In [18]:
save_json(dataset, r'D:/Projects/Test/IEEE_papers/processed_data/processed_data.json')

In [19]:
def call_gpt(messages, model="gpt-3.5-turbo-0613"):
    completions = openai.ChatCompletion.create(
        model=model,
        n=1,
        stop=None,
        temperature = 0.0,
        messages=messages)
    gpt_response = completions['choices'][0]['message']['content'].strip() 
    return gpt_response   

In [20]:
def merge_sentences(datum_sentences):
    sentence_list = [" ".join(sentence_word_list) for sentence_word_list in datum_sentences] # merge the words into sentences
    paragraph = " ".join(sentence_list)
    return paragraph


In [52]:
def extract_events(sentence):
    messages = [
        { 
            "role": "system", 
            "content": """
                You are a state of the art event extraction system. 
                Your task is to extract only the most important event that describes the main research idea from Research paper abstracts. 
                Strictly extract only one event. This event should be the major research focus of the abstract.
                The events should be human-readable. 
                Reply in JSON format with each line being an event in the format:
                [event];
                The abstract of research papers will be provided by the user.
            """
        },
        { "role": "system", "name": "example_user", "content": "We present KiriPhys, a new type of data physicalization based on kirigami, a traditional Japanese art form that uses paper-cutting. Within the kirigami possibilities, we investigate how different aspects of cutting patterns offer opportunities for mapping data to both independent and dependent physical variables. As a first step towards understanding the data physicalization opportunities in KiriPhys, we conducted a qualitative study in which 12 participants interacted with four KiriPhys examples. Our observations of how people interact with, understand, and respond to KiriPhys suggest that KiriPhys: 1) provides new opportunities for interactive, layered data exploration, 2) introduces elastic expansion as a new sensation that can reveal data, and 3) offers data mapping possibilities while providing a pleasurable experience that stimulates curiosity and engagement."},
        { "role": "system", "name": "example_system", "content": "[data physicalization based on kirigami];"},       
        { "role": "user", "content": sentence}
    ]
    events = call_gpt(messages)
    return events

In [53]:
def merged(sentence,cl):
    messages = [
        { 
            "role": "system", 
            "content": """
                You are a named entity recognition model. You will be given an abstract of a research paper and the event recognized in that abstract by the user.
                The format of the input defining event in research paper will be:
                [event];
                Extract the 2 main participants in the event. A participant can be a proposed solution, product, algorithm, technique, instruments, tools, processes or phenomena.  
                The number of main participants should be 2 or less strictly. 
                Ignore any other participants other than the two main participants.
                Reply in JSON format with each line being an event in the format:
                [event]:[main participant 1],[main participant 2];
            """
        },   
        { "role": "system", "name": "example_user", "content": "We present KiriPhys, a new type of data physicalization based on kirigami, a traditional Japanese art form that uses paper-cutting. Within the kirigami possibilities, we investigate how different aspects of cutting patterns offer opportunities for mapping data to both independent and dependent physical variables. As a first step towards understanding the data physicalization opportunities in KiriPhys, we conducted a qualitative study in which 12 participants interacted with four KiriPhys examples. Our observations of how people interact with, understand, and respond to KiriPhys suggest that KiriPhys: 1) provides new opportunities for interactive, layered data exploration, 2) introduces elastic expansion as a new sensation that can reveal data, and 3) offers data mapping possibilities while providing a pleasurable experience that stimulates curiosity and engagement."},
        { "role": "system", "name": "example_system", "content": "[data physicalization based on kirigami],[Kirigami],[KiriPhys];"},    
        { "role": "user", "content": f"This is the input defining the extracted event :{cl}"},  
        { "role": "user", "content": f"This is the news article:{sentence}"},
    ]
    events = call_gpt(messages)
    return events

In [69]:
def merged_all(sentence,cl):
    messages = [
        { 
            "role": "system", 
            "content": """
                You will be given an abstract of a research paper and the event graph of one event in that article by the user.
                The format of the event graph will be:
                [event]:[main participant 1],[main participant 2];
                Arguments refer to the specific entities, participants, or elements that play specific roles in an event.
                Extract the main arguments and their roles involved in each event.
                Strictly assign an argument type to each argument.                
                Reply in JSON format with each line being an event in the format:
                [event]:[main participant 1],[main participant 2]:[argument type 1 - argument 1],[argument type 2 - argument 2],...; 
            """
        },   
        { "role": "system", "name": "example_user", "content": "We present KiriPhys, a new type of data physicalization based on kirigami, a traditional Japanese art form that uses paper-cutting. Within the kirigami possibilities, we investigate how different aspects of cutting patterns offer opportunities for mapping data to both independent and dependent physical variables. As a first step towards understanding the data physicalization opportunities in KiriPhys, we conducted a qualitative study in which 12 participants interacted with four KiriPhys examples. Our observations of how people interact with, understand, and respond to KiriPhys suggest that KiriPhys: 1) provides new opportunities for interactive, layered data exploration, 2) introduces elastic expansion as a new sensation that can reveal data, and 3) offers data mapping possibilities while providing a pleasurable experience that stimulates curiosity and engagement."},
        { "role": "system", "name": "example_system", "content": "[data physicalization based on kirigami]:[Kirigami],[KiriPhys]:[type of data physicalization - KiriPhys],[art form - Kirigami];"},   
        { "role": "user", "content": f"This is the input defining the extracted event and the main participants, strictly use only this event for further tasks  :{cl}"},  
        { "role": "user", "content": f"This is the news article:{sentence}"},
    ]
    events = call_gpt(messages)
    return events
                # For each event assign roles that the main participants play.
                # Only assign roles to the main participants given by user input. 
                # These should be the participants heavily involved in the event.

In [55]:
def strip_sentence(sentence):
    if sentence.startswith('The article discussed how'):
        stripped_sentence = sentence.replace('The article discussed how', '').strip()
        stripped_sentence = re.sub(",","",stripped_sentence)
    elif sentence.startswith('The article discussed'):
        stripped_sentence = sentence.replace('The article discussed', '').strip()
        stripped_sentence = re.sub(",","",stripped_sentence)
    else:
        print("!!!")
    return stripped_sentence

In [25]:
# RAMS
saved_dataset = []
for datum in dataset:
    saved_datum = {}
    article = merge_sentences(datum['sentences'])
    arguments = (article)
  #  sentence = summarize_sentence(article, arguments)
  #  print(sentence)
    # saved_datum['content'] = datum['sentences']
    # saved_datum['url'] = datum['source_url']
    # saved_datum['summary'] = sentence
    saved_dataset.append(arguments)
save_json(saved_dataset, r'D:/Projects/Test/summarized/args.json')

KeyError: 'sentences'

In [72]:
dat = json.load(open(r'../IEEE_papers/processed_data/processed_data.json'))
# cl = json.load(open(r'D:/Projects/Test/All the News/events/merged_bb.json'))
#ar = json.load(open(r'D:/Projects/Test/All the News/events/cl_ents.json'))
res_events = []
error_datum = []
# sentence = "The article discussed the extensive history of doping in Russia, dating back to the 1983 Soviet Union's detailed instructions to inject top athletes with anabolic steroids in order to ensure dominance at the Los Angeles Olympics. Dr. Sergei Portugalov, a key figure in Russia's current doping scandal, was named as the mastermind behind the doping program. The revelations of these schemes led to the banning of Russia's track and field team from the Rio Games, the most severe doping penalty in Olympic history."
for index, datum in enumerate(dat):
    print('{}/{}'.format(index, len(dat)))
    sentence = datum['Abstract'].strip()
    sentence = re.sub("-"," ",sentence)
    cl = extract_events(sentence)
    # cl = re.sub(":","",cl)
    # cl = re.sub("-","",cl)
    events = merged(sentence,cl)
    # print(events)
    ev = merged_all(sentence,events)
    datum['events'] = ev
    # print(ev)
    res_events.append(datum)
save_json(res_events, r'../IEEE_papers/Events/events_merged_final.json')

0/20
1/20
2/20
3/20
4/20
5/20
6/20
7/20
8/20
9/20
10/20
11/20
12/20
13/20
14/20
15/20
16/20
17/20
18/20
19/20


In [66]:
AllTheNews_summarized = json.load(open(r'D:/Projects/Test/All the News/data/summarized/summary.json'))
# cl = json.load(open(r'D:/Projects/Test/All the News/events/classes.json'))
res_events = []
error_datum = []
for index, datum in enumerate(AllTheNews_summarized):
    print('{}/{}'.format(index, len(AllTheNews_summarized)))
    sentence = strip_sentence(datum['summary'])
    cl = extract_cl(sentence)
    events = merged(sentence,cl)
    # datum['events'] = events
    res_events.append(events)
save_json(res_events, r'D:/Projects/Test/All the News/events/ev.json')

0/20
1/20
2/20
3/20
4/20
5/20
6/20
7/20
8/20
9/20
10/20
11/20
12/20
13/20
14/20
15/20
16/20
17/20
18/20
19/20


In [73]:
from string import punctuation
def post_process_events(dataset):
    for index, datum in enumerate(dataset):
        datum['doc_id'] = index
        datum['events_raw'] = datum['events']
        events_str = datum['events'].split('\n')
        events = []
        for event_str in events_str:
            arguments=[]
            main_characters = []
            event_str = event_str.strip()
            components = event_str.split(':')
            event_type = components[0].strip()
            # event_type_p=event_type_raw[0].split('-')
            # event_type = event_type_raw[0][1:]
            # trigger = event_type_raw[1:2]
            # trigger=' '.join([str(elem) for elem in trigger]).strip().strip(punctuation)
            # if(trigger==""):
            #     trigger=event_type
            chars = components[1].split('],[')
            # arguments_raw = [arg.strip().strip(punctuation) for arg in components[1:]]
            arguments_raw = components[2:]
            # print(arguments_raw)
            # chars = re.sub(",","",chars)
            for dat in chars:
                dat=re.sub(",","",dat)
                dat = str(dat).replace('[', '').replace(']', '')
                # print(dat)
                main_characters.append(dat)
            for args in arguments_raw:
                # args = re.sub(",","",args)
                temp = args.split('],[')
                for arg in temp:
                    # print(arg)
                    # arg=re.sub(",","",arg)
                    arg_raw=arg.split('-')
                    arg_type=arg_raw[0].strip()
                    # arg_type=re.sub(",","",arg_type)
                    args = arg_raw[1:]
                    args_final=' '.join([str(elem) for elem in args]).strip().strip(punctuation)
                    # args_final=re.sub(",","",args_final)
                    arg_type = str(arg_type).replace('[', '').replace(']', '')
                    arguments.append({arg_type:args_final})
            events.append({'Trigger':event_type, 'Main Participants': main_characters, 'Arguments': arguments})
        datum['events'] = events
    return dataset

dataset = json.load(open(r'../IEEE_papers/Events/events_merged_final.json'))
processed_dataset = post_process_events(dataset)
save_json(processed_dataset, r'../IEEE_papers/Result/events_merged_final.json')
